# yolo 깃허브 clone, 필요 파일 설치

In [ ]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
!pip install -qr requirements.txt  
!pip install -q roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 15002, done.
remote: Total 15002 (delta 0), reused 0 (delta 0), pack-reused 15002
Receiving objects: 100% (15002/15002), 14.08 MiB | 9.97 MiB/s, done.
Resolving deltas: 100% (10285/10285), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.2 MB/s eta 0:00:00


In [ ]:

import os, yaml, glob, torch
from distutils.dir_util import copy_tree
from IPython.core.magic import register_line_cell_magic
from roboflow import Roboflow

# 데이터셋 다운로드 

In [ ]:
%cd /content/yolov5
#칼 데이터셋 다운로드
rf = Roboflow(api_key="krcQlWTnG5PUWdI1yzWt")
project = rf.workspace("augustus").project("knife_dataset-kysbg")
dataset = project.version(2).download("yolov5")
#담배 데이터셋 다운로드
project = rf.workspace("cigarettesmokingdetection").project("cigarette-detection-uyqvc")
dataset = project.version(1).download("yolov5")

#얼굴 데이터셋 다운로드
project = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
dataset = project.version(20).download("yolov5")

/content/yolov5
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to knife_dataset-2 in yolov5pytorch:: 100%|██████████| 4413/4413 [00:01<00:00, 3692.83it/s]

loading Roboflow workspace...


loading Roboflow project...


Extracting Dataset Version Zip to Cigarette-detection-1 in yolov5pytorch:: 100%|██████████| 4548/4548 [00:01<00:00, 3239.96it/s]

loading Roboflow workspace...


loading Roboflow project...


Extracting Dataset Version Zip to Face-Detection-20 in yolov5pytorch:: 100%|██████████| 2750/2750 [00:01<00:00, 2300.68it/s]


# train, val, test 데이터 경로 설정

In [ ]:
#옮길 데이터셋과 옮겨질 위치의 데이터셋 이름
from_data = 'Cigarette-detection-1'
to_data = 'knife_dataset-2'

#해당 데이터셋에 특정 클래스가 몇개 있는지 확인

to_class = '0'
from_class = '2'

train = glob.glob(f'/content/yolov5/{from_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{from_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{from_data}/test/labels/*.txt')
count=0
for file_path in train+valid+test:
    with open(file_path,'r') as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].find(to_class, 0,1) == 0:
          count+= 1

print('From:',count)

train = glob.glob(f'/content/yolov5/{to_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{to_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{to_data}/test/labels/*.txt')
count=0
for file_path in train+valid+test:
    with open(file_path,'r') as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].find(from_class, 0,1) == 0:
          count+= 1
print('To:',count)

From: 2496
To: 0


In [ ]:
#from data에 있는 from_target_class 번호를 to_target_class로 변경

#만약 변경해야할 클래스수가 많다면 위에서부터 진행

train = glob.glob(f'/content/yolov5/{from_data}/train/labels/*.txt')
valid = glob.glob(f'/content/yolov5/{from_data}/valid/labels/*.txt')
test = glob.glob(f'/content/yolov5/{from_data}/test/labels/*.txt')
for file_path in train+valid+test:
    with open(file_path,'r') as f:
        lines = f.readlines()
    s = ''
    for i in range(len(lines)):
        s += lines[i].replace(to_class,from_class,1) +'\n' 
    with open(file_path,'w') as f:
        f.write(s)

In [ ]:
# 셀 통합
copy_tree(f'/content/yolov5/{from_data}/',f'/content/yolov5/{to_data}/')

# data.yaml 파일 설정

In [ ]:
#데이터를 합치고 난 후 class 수(nc) 각 class별 이름(names) 설정 
dataset.location = '/content/yolov5/knife_dataset-2'
%cat {dataset.location}/data.yaml

names: ['knife', 'face', 'smoking']
nc: 3
train: ./knife_dataset-2/train/images
val: ./knife_dataset-2/valid/images
test: ./knife_dataset-2/test/images


In [ ]:
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = yaml.safe_load(stream)['nc']

In [ ]:
#모델 설정
%cat /content/yolov5/models/yolov5m.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5m.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# train

In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 32 --epoch 10 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5m.yaml --weights yolov5m.pt --name mo_yolov5m_results

/content/yolov5
train: weights=yolov5m.pt, cfg=./models/custom_yolov5m.yaml, data=/content/yolov5/knife_dataset-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=mo_yolov5m_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 436 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.13.1+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.